In [15]:
!pip install mindspore

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple/

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [16]:
# 引入库
!pip install cvxopt
import numpy as np
import pandas as pd
import cvxopt
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple/

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [17]:
# 数据标准化
def generalization(x):
    num_feature = x.shape[1]
    for i in range(num_feature):
        x[:, i] = (x[:, i] - min(x[:, i])) / (max(x[:, 1]) - min(x[:, i]))
    return x

In [18]:
# 训练集测试集分割
def train_test_split(x, y, test_rate, random_state=0):
    if random_state is not None:
        random.seed(random_state)
    # num_x = x.shape[0]
    # 三组分别打乱
    x1 = X[0:50, :]
    x2 = X[50:100, :]
    x3 = X[100:, :]
    y1 = y[0:50]
    y2 = y[50:100]
    y3 = y[100:]
    num_x = x1.shape[0]
    # 类别内部打乱
    index = list(range(num_x))
    random.shuffle(index)
    x1 = x1[index]
    y1 = y1[index]
    x2 = x2[index]
    y2 = y2[index]
    x3 = x3[index]
    y3 = y3[index]

    # 分割
    split = round(num_x * test_rate)
    x1_test = x1[:split, :]
    x1_train = x1[split:, :]
    y1_test = y1[:split]
    y1_train = y1[split:]
    x2_test = x2[:split, :]
    x2_train = x2[split:, :]
    y2_test = y2[:split]
    y2_train = y2[split:]
    x3_test = x3[:split, :]
    x3_train = x3[split:, :]
    y3_test = y3[:split]
    y3_train = y3[split:]

    x_train = np.vstack((x1_train, x2_train, x3_train))
    x_test = np.vstack((x1_test, x2_test, x3_test))
    y_train = np.hstack((y1_train, y2_train, y3_train))
    y_test = np.hstack((y1_test, y2_test, y3_test))

    # 整体打乱
    index1 = list(range(x_train.shape[0]))
    random.shuffle(index1)
    x_train = x_train[index1]
    y_train = y_train[index1]

    index2 = list(range(x_test.shape[0]))
    random.shuffle(index2)
    x_test = x_test[index2]
    y_test = y_test[index2]

    return x_train, x_test, y_train, y_test

In [19]:
# Iris数据集读取
# 150个样本，4维特征，三种类别
iris = pd.read_csv('iris.data', header=None) # 直接在同文件夹下读取iris数据集
iris1 = pd.read_csv('iris.data', header=None)
iris2 = pd.read_csv('iris.data', header=None)
# iris3 = pd.read_csv('iris.data', header=None)
iris = np.array(iris)
iris1 = np.array(iris1)
iris2 = np.array(iris2)
# iris3 = np.array(iris3)
X = iris1[:, 0:4]
X = np.float64(X)
y = iris[:, 4]

# 用于对比的y标签
for i in range(len(y)):
    if y[i] == 'Iris-setosa':
        y[i] = 0
    elif y[i] == 'Iris-versicolor':
        y[i] = 1
    else:
        y[i] = 2
y = np.float64(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_rate=0.2, random_state=32)
print('y_test =', y_test)

y_test = [0. 1. 1. 2. 0. 1. 2. 1. 1. 0. 0. 2. 2. 1. 2. 1. 0. 0. 1. 2. 0. 2. 2. 0.
 2. 2. 0. 1. 1. 0.]


In [20]:
# 对读入的数据集进行分割操作
'''制作3个SVM分类器所需的训练集和测试集'''
# Iris-setosa 为 +1
# X1 = np.float64(iris[0:100, 0:4])
y1 = iris1[:, 4]
for i in range(len(y1)):
    if y1[i] == 'Iris-setosa':
        y1[i] = 1
    else:
        y1[i] = -1
y1 = np.int64(y1)
X1_train, X1_test, y1_train, y1_test = train_test_split(X, y1, test_rate=0.2, random_state=32)

# Iris-versicolor  为 +1
# X2 = np.float64(iris[0:100, 0:4])
y2 = iris2[:, 4]
for i in range(len(y2)):
    if y2[i] == 'Iris-versicolor':
        y2[i] = 1
    else:
        y2[i] = -1
y2 = np.int64(y2)
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y2, test_rate=0.2, random_state=32)

In [21]:
# 定义三种常见的核函数
# 线性核
def linear_kernal(**kwargs):
    def K(x1, x2):
        return np.inner(x1, x2)

    return K


# 多项式核
def polynomial_kernal(power, coef, **kwargs):
    def K(x1, x2):
        return (np.inner(x1, x2) + coef) ** power

    return K


# 径向基核
def rbf_kernal(gamma, **kwargs):
    def K(x1, x2):
        distance = np.linalg.norm(x1 - x2) ** 2
        return np.exp(-gamma * distance)

    return K

In [22]:
# 定义SVM类
class svm:
    # 输入参数
    # param kernal:核函数
    # param penaltyC:软间隔惩罚项C
    # param power, gamma,coef为核函数的一些参数
    def __init__(self, kernal=linear_kernal, penaltyC=1, power=1, gamma=1, coef=1):
        self.kernal = kernal
        self.penaltyC = penaltyC
        self.power = power
        self.gamma = gamma
        self.coef = coef
        self.kernal = self.kernal(
            power=self.power,
            gamma=self.gamma,
            coef=self.coef)

    def train(self, x, y):
        x_num = x.shape[0]
        kernal_matrix = self.kernal(x, x) + (1 / self.penaltyC) * np.eye(x_num)

        # 计算标准凸二次规划的几个参数
        p = cvxopt.matrix(kernal_matrix * np.outer(y, y))
        q = cvxopt.matrix(-np.ones([x_num, 1], np.float64))
        g = cvxopt.matrix(-np.eye(x_num))
        h = cvxopt.matrix(np.zeros([x_num, 1], np.float64))

        y = np.float64(y)
        a = cvxopt.matrix(y, (1, x_num))
        b = cvxopt.matrix(0.)

        # 使用凸规划cvxopt求解SVM目标函数（算lagrange乘子 alpha）
        cvxopt.solvers.options['show_progress'] = False
        solution = cvxopt.solvers.qp(p, q, g, h, a, b)
        alpha = np.float32(np.array(solution['x']))
        alpha[alpha <= 1e-4] = 0

        # 求权重w和截距b
        w = np.sum(np.reshape(y, [-1, 1]) * alpha * x, axis=0)
        b = np.mean(np.reshape(y, [-1, 1]) - np.reshape(np.dot(w, np.transpose(x)), [-1, 1]))
        self.w = w
        self.b = b

        return w, b, alpha

    def predict(self, x_test):
        y_predict = []
        for sample in x_test:
            predict1 = self.kernal(self.w, sample) + self.b
            predict1 = np.int64(np.sign(predict1))
            y_predict.append(predict1)
        #         y_predict.tolist()
        return y_predict

In [24]:
if __name__ == '__main__':
    svm1 = svm(linear_kernal, penaltyC=1)
    w1, b1, alpha1 = svm1.train(X1_train, y1_train)
    y1_predict = svm1.predict(X1_test)
    print('Iris1_accuracy:', accuracy_score(y1_test, y1_predict))

    svm2 = svm(linear_kernal, penaltyC=1)
    w2, b2, alpha2 = svm2.train(X2_train, y2_train)
    y2_predict = svm2.predict(X2_test)
    print('Iris2_accuracy:', accuracy_score(y2_test, y2_predict))

    # svm3 = svm(linear_kernal, penaltyC=1)
    # w3, b3, alpha3 = svm3.train(X3_train, y3_train)
    # predict3, y3_predict = svm3.predict(X3_test)
    # print('Iris3_accuracy:', accuracy_score(y3_test, y3_predict))
    
    # predict_seq = np.float64(predict_seq)
    # print('predict_seq =', predict_seq)

    predict_seq = []
    for i in range(len(y1_predict)):
        if y1_predict[i] >= 0:
            predict_seq.append(0)
        elif y2_predict[i] >= 0:
            predict_seq.append(1)
        else:
            predict_seq.append(2)
    print('Iris_accuracy:', accuracy_score(y_test, predict_seq))
    print('Iris_recall:', recall_score(y_test, predict_seq, average='micro'))

Iris1_accuracy: 1.0
Iris2_accuracy: 0.8666666666666667
Iris_accuracy: 0.8666666666666667
Iris_recall: 0.8666666666666667
